<a href="https://colab.research.google.com/github/rocioparra/redes-neuronales/blob/master/03-clasificacion-texto/Ejercicio_Clasificaci%C3%B3n_Texto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Estos dos comandos evitan que haya que hacer reload cada vez que se modifica un paquete
%load_ext autoreload
%autoreload 2

# Ejercicio de clasificación de texto

Naive Bayes es una técnica estadística que consiste en repetir el método anterior en problemas cuyos sucesos no son independientes, pero suponiendo independencia.
A lo largo de este trabajo desarrollarán un modelo de Naive Bayes para el problema de clasificación de artículos periodístios.En este caso podemos estimar la probabilidad de ocurrencia de cada palabra según la categoría a la que pertenece el artículo.

## Dataset


El primer paso es obtener el dataset que vamos a utilizar. El dataset a utilizar es el de TwentyNewsGroup(TNG) que está disponible en sklearn.

Se puede encontrar más información del dataset en la documentación de scikit-learn.

In [2]:
from sklearn.datasets import fetch_20newsgroups
from helper import autosave

# autosave asegura que se guarden los datos en un archivo
@autosave('twenty-md={metadata}-{subset}')
def get_20newsgroup(subset='train', metadata=False):
    if metadata:
        return fetch_20newsgroups(subset=subset, shuffle=True)
    else:
        # de acuerdo a recomendacion de sklearn, no considerar metadata
        # para obtener resultados mas representativos
        return fetch_20newsgroups(subset=subset, remove=('headers', 'footers', 'quotes'), shuffle=True)

In [ ]:
#Loading the data set - training data.

twenty_train = get_20newsgroup(subset='train', metadata=False)

El siguiente paso es analizar el contenido del dataset, como por ejemplo la cantidad de artículos, la cantidad de clases, etc.

### Preguntas

1) ¿Cuántos articulos tiene el dataset?

In [ ]:
len(twenty_train.data)

2) ¿Cuántas clases tiene el dataset?

In [ ]:
len(twenty_train.target_names)

3) ¿Es un dataset balanceado?

In [ ]:
import numpy as np
_, counts = np.unique(twenty_train.target, return_counts=True)
if len(set(counts)) == 1:
    print('El dataset está balanceado')
else:    
    print('El dataset no está balanceado')

4) ¿Cuál es la probabilidad a priori de la clase 5? A que corresponde esta clase?

In [ ]:
priori5 = counts[5]/sum(counts)
print(f'La clase 5 ({twenty_train.target_names[5]}) tiene probabilidad a priori {priori5:3}')

5) ¿Cuál es la clase con mayor probabilidad a priori?

In [ ]:
max_class = np.argmax(counts)
print(f'La clase {max_class} ({twenty_train.target_names[max_class]}) tiene la máxima probabilidad a priori, {counts[max_class]/sum(counts):.2}')

## Preprocesamiento

Para facilitar la comprensión de los algoritmos de preprocesamiento, se aplican primero a un solo artículo.


Mas info en:
http://text-processing.com/demo/stem/

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from helper import print_list


nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
stemmer = PorterStemmer()

article = twenty_train.data[0]
article

- **Tokenization (nltk):**

In [ ]:
tok = word_tokenize(article)
print_list(tok)
# print feo porque me ponia enters y quedaba muy largo

- **Lemmatization (nltk):**

In [ ]:
lemmatizer = WordNetLemmatizer()
lem=[lemmatizer.lemmatize(x, pos='v') for x in tok]
print_list(lem)

- **Stop Words (nltk):**


In [ ]:
stop = [x for x in lem if x not in stopwords.words('english')]
print_list(stop)

In [ ]:
print(len(stopwords.words('english')))

- **Stemming (nltk):**


In [ ]:
stem = [stemmer.stem(x) for x in stop]
print_list(stem)

- **Filtrado de palabras:**


In [ ]:
alpha = [x for x in stem if x.isalpha()]
print_list(alpha)

### Preprocesamiento completo

Utilizar o no cada uno de los métodos vistos es una decisión que dependerá del caso particular de aplicación. Para este ejercicio vamos a considerar las siguientes combinaciones:

- Tokenización
- Tokenización, Lematización, Stemming.
- Tokenización, Stop Words.
- Tokenización, Lematización, Stop Words, Stemming.
- Tokenización, Lematización, Stop Words, Stemming, Filtrado.

In [ ]:
def filter_article(article, filts):
    filts = filts.split()

    if 'lem' in filts:
        article = [lemmatizer.lemmatize(x,pos='v') for x in article]

    if 'stop' in filts:
        article = [x for x in article if x not in stopwords.words('english')]

    if 'stem' in filts:
        article = [stemmer.stem(x) for x in article]

    if 'filt' in filts:
        article = [x for x in article if x.isalpha()]
    
    return article


@autosave(fmt='{name}-{filts}')
def filter_articles(name, articles, filts):
    
    filtered_articles = []
    for data in articles:
        tok = word_tokenize(data)
        filtered_articles.append(filter_article(tok, filts))
            
    return filtered_articles

In [ ]:
ans_fmt = """Preprocesamiento: {preproc}
Longitud del vocabulario: {vocab_len}
"""
preprocessing = ['tok', 'tok lem stem', 'tok stop', 'tok lem stop stem', 'tok lem stop stem filt']

    
for preproc in preprocessing:
    filtered_articles = filter_articles('train-nometadata', twenty_train.data, preproc)
    vocab = set([word for article in filtered_articles for word in article])
    print(ans_fmt.format(preproc=preproc, vocab_len=len(vocab)))

### Preguntas

- **Cómo cambia el tamaño del vocabulario al agregar Lematización y Stemming?**

Sólo con tokenización: 161698 palabras

Con lematización y stemming: 126903 palabras (21.5% menor, 34795 palabras menos).

El vocabulario se reduce no porque la cantidad total de palabras sea menor, sino porque palabras que originalmente eran distintas ahora son iguales (por ejemplo, "is" y "was" se convierten ambas en "be").

La reducción del 20% sugiere que de cada 5 palabras, dos son dos "versiones" de la misma, lo cual es razonable si se tiene en cuenta que es muy común usar una misma palabra en singular y plural en un mismo contexto ("car" / "cars"), adverbios y adjetivos con la misma raíz ("real" / "really"), pronombres en distintos casos ("I" / "me"), etcétera (en español esto probablemente sería incluso más pronunciado, al haber mayor cantidad de declinaciones verbales distintas, y tener géneros para adjetivos y artículos).

- **Cómo cambia el tamaño del vocabulario al Stop Words?**

Sólo con tokenización: 161698 palabras

Con stop words: 161533 palabras (0.1% menor, 165 palabras menos)

El vocabulario se reduce porque se remueven las palabras contenidas en el conjunto de stop words. Es razonable entonces que la reducción del vocabulario sea menor de este caso, ya que como máximo se podrán remover tantas palabras como haya en la lista de stop words (en este caso, 179).

- **Analice muy brevemente ventajas y desventajas del tamaño del dataset en cada caso.**

Para el caso de stopwords, es útil porque es razonable pensar que ese conjunto de palabras estará presente en todos los artículos, y por lo tanto no aportará demasiada información. Por ejemplo, una palabra como "car", "player" o "God" nos da más información sobre de qué se está hablando que "in", "no" o "is".

Sin embargo, también pueden imaginarse casos donde esto no sea cierto. Por ejemplo, si una clase se caracteriza por narraciones en primera persona, mientras que los demás suelen ser más impersonales, se estaría ignorando la información que aportan palabras como "I", "me", etc. 

En cuanto a la lematización y stemming, puede hacerse un análisis muy similar. En muchos casos, es muy útil combinar las probabilidades de palabras similares: en un artículo de deportes, las palabras "match" y "matches" apuntan a un contexto similar. Lo mismo puede decirse en cuanto a el tiempo de los verbos: no es particularmente relevante para determinar si se está hablando de deportes, ya que podría estarse especulando sobre o anunciando eventos que sucederán en el futuro ("if they win"), o relatando eventos del pasado ("after they won"). Por lo tanto, tratar ambos casos como idénticos podría ser beneficioso.

Sin embargo, nuevamente pueden pensarse casos donde esto no será cierto: puede que una clase se caracterice por hablar sobre eventos del pasado, mientras que otra se dedique más a explicaciones en presente, y ese matiz se perdería con lematización y stemming.

## Vectorización de texto

- **Obtención del vocabulario y obtención de la probabilidad**

Como se vió en clase, los vectorizadores cuentan con dos parámetros de ajuste.

- max_df: le asignamos una maxima frecuencia de aparición, eliminando las palabras comunes que no aportan información.

- min_df: le asignamos la minima cantidad de veces que tiene que aparecer una palabra.


## Entrenamiento del modelo

Primero deben separar correctamente el dataset para hacer validación del modelo.
Y luego deben entrenar el modelo de NaiveBayes con el dataset de train.

Deben utilizar un modelo de NaiveBayes Multinomial y de Bernoulli. Ambos modelos estan disponibles en sklearn.

Finalmente comprobar el accuracy en train.

In [ ]:
@autosave(fmt='{name}-joined-{filts}')
def get_filtered_joined_articles(name, articles, filts):
    articles = filter_articles(name, articles, filts)
    for i in range(len(articles)):
        articles[i] = ' '.join(articles[i])
        
    return articles

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
import pandas as pd
import os.path


OUT_FILE = 'out-nometadata4.csv'

vectorizers = [CountVectorizer, TfidfVectorizer]
classifiers = [MultinomialNB, BernoulliNB]
max_dfs = [.05, 0.1, 0.25, 0.3, 0.5, 0.75, 0.8, 0.95]
min_dfs = [1, 2, 5, 10]

results = []

import pandas as pd


total_its = len(vectorizers)*len(classifiers)*len(min_dfs)*len(max_dfs)*len(preprocessing)
i = 1

if os.path.isfile(OUT_FILE):
    results = pd.read_csv(OUT_FILE)
else:
    results = []

    for filts in preprocessing:
        data = get_filtered_joined_articles(name='train-nometadata', filts=filts) 
        
        for max_df in max_dfs:
            for min_df in min_dfs:
                
                for Vectorizer in vectorizers:
                    count_vect = Vectorizer(max_df=max_df, min_df=min_df)
                    raw_data = count_vect.fit_transform(data) 

                    for Classifier in classifiers:
                        print(f'Computing: F={filts}, M={max_df}, m={min_df}, V={Vectorizer.__name__}, C={Classifier.__name__} ({i}/{total_its})...')
                        i += 1
                        
                        clf = Classifier()
                        clf.fit(raw_data, twenty_train.target)
                        score = clf.score(raw_data, twenty_train.target)

                        results.append({
                            'max_df': max_df,
                            'min_df': min_df,
                            'filts': filts,
                            'vectorizer': Vectorizer.__name__,
                            'model': Classifier.__name__,
                            'score': score
                        })
                        
                        print(f'Score: {score:.5%}')
                        print(50 * '-')
    results = pd.DataFrame(results)
    results.to_csv(OUT_FILE, index=False)

### Preguntas

- **¿Con que modelo obtuvo los mejores resultados? Explique por qué cree que fue así.**

In [ ]:
best_hyper = best_hypers.iloc[0]

In [ ]:
best = results['score'].max()
best_filt = results['score']==best
best_hypers = results[best_filt]
best_hypers

Los mejores resultados se obtenieron utilizando:
- clasificador multinomial
- vectorizador TF-IDF, con min_df=1 y max_df=0.05
- preprocesamiento: únicamente tokenización

No se disminuyó más el valor de max_df para evitar overfitting (quedarse sólo con palabras que estén en uno o dos artículos, y por lo tanto poder determinar la clase casi determinísticamente para el set de train, pero probablemente obteniendo un modelo no muy útil para analizar otros sets de datos).

El valor de max_df=0.05 sugiere que se eliminan del set palabras que sean comunes en más de una clase (si bien el set no está exactamente balanceado, se vio previamente que la máxima probabilidad es de 0.053, y si fuese 0.05 todas las clases tendrían la misma). Con min_df=1 no se descarta ninguna palabra que aparezca en al menos un artículo, y utilizar solo tok hace que las palabras queden tal como estaban. Considerando la naturaleza especializada de las clases, y lo distintas que son entre sí (ateísmo, hardware IBM, MS Windows,  armas, política en el medio oriente, espacial, cripto...), probablemente lo que esté sucediendo es que se conserva sólo un vocabulario lo suficientemente especializado como para detectar con la mayor certeza posible a cuál de las 20 clases pertenece.

Esto es consistente con que los mejores resultados se obtengan con TF-IDF, ya que de esta manera se está penalizando a palabras que son comunes entre muchos artículos, nuevamente haciendo foco en vocabulario especializado.

In [ ]:
results.sort_values(by=['score'], ascending=False, inplace=True)

In [ ]:
len(results)

In [ ]:
results[results.model=='MultinomialNB'].sort_values(by='score', ascending=True).head()

In [ ]:
results[results.model=='BernoulliNB'].head()

Se obtuvieron mejores resultados con el modelo multinomial. De las 640 combinaciones de hiperparámetros testeadas, absolutamente todas las pruebas con el modelo de Bernoulli arrojaron peores resultados que el peor scoring obtenido para multinomial (0.6828 para el mejor caso de Bernoulli, y 0.7235 para el peor de multinomial).

Este resultado es el esperado, ya que el modelo multinomial representa más fielmente los datos de este problema en particular: la ocurrencia de una palabra en un artículo aporta mucha menos información si se considera binariamente (palabra está presente / no está presente en el artículo). Con el modelo de Bernoulli, en este caso, se está perdiendo información.

## Performance de los modelos

En el caso anterior, para medir la cantidad de artículos clasificados correctamente se utilizó el mismo subconjunto del dataset que se utilizó para entrenar.

Esta medida no es una medida del todo útil, ya que lo que interesa de un clasificador es su capacidad de clasificación de datos que no fueron utilizados para entrenar. Es por eso que se pide, para el clasificador entrenado con el subconjunto de training, cual es el porcentaje de artículos del subconjunto de testing clasificados correctamente. Comparar con el porcentaje anterior y explicar las diferencias.

Finalmente deben observar las diferencias y extraer conclusiones en base al accuracy obtenido, el preprocesamiento y vectorización utilizado y el modelo, para cada combinación de posibilidades.

In [ ]:
#Loading the data set - training data.
twenty_test = get_20newsgroup(subset='test', metadata=False)

In [ ]:
train_data = get_filtered_joined_articles(name='train-nometadata', 
                                        articles=twenty_train.data,
                                        filts=best_hyper.filts)
test_data = get_filtered_joined_articles(name='test-nometadata', 
                                        articles=twenty_test.data,
                                        filts=best_hyper.filts)

In [ ]:
count_vect = eval(best_hyper.vectorizer)(max_df=best_hyper.max_df, min_df=best_hyper.min_df)
raw_data_train = count_vect.fit_transform(train_data) 
raw_data_test = count_vect.transform(test_data) 

In [ ]:
clf = eval(best_hyper.model)()
clf.fit(raw_data_train, twenty_train.target)
score = clf.score(raw_data_test, twenty_test.target)
score

### Preguntas

- **El accuracy en el dataset de test es mayor o menor que en train? Explique por qué.**

La accuracy es menor en test que en train porque los parámetros del modelo son los estimados a partir de train, y por lo tanto está garantizado que dado un set de hiperparámetros y un modelo dado, será la mejor representación de los datos posible. Esto no es cierto con test - podría darse que en algún caso en particular, la información de test se ajuste a la obtenida a partir de train, incluso mejor que el mismo train, pero esto no es una garantía (y no es lo que se espera, ni en lo que en la mayoría de los casos ocurre).